### Manual model run
This notebook does a single manual run of the model from the default parameters in ./inputs/parameters.yml and produces some basic plots, including the matrix scaling, which is not influenced by calibration to a significant extent.

In [ ]:
from aust_covid.inputs import get_ifrs
from aust_covid.model import build_model
from aust_covid.calibration import get_targets
from aust_covid.plotting import plot_single_run_outputs, plot_example_model_matrices
from inputs.constants import PROJECT_PATH, SUPPLEMENT_PATH
from aust_covid.utils import add_image_to_doc
from emutools.tex import StandardTexDoc
from emutools.parameters import load_param_info

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'model_construction', 'Model construction', 'austcovid')
targets = get_targets(app_doc)
param_info = load_param_info()
param_info['value'].update(get_ifrs(app_doc))
parameters = param_info['value'].to_dict()

In [ ]:
epi_model = build_model(app_doc, mobility_sens=True)

In [ ]:
epi_model.run(parameters=parameters)

In [ ]:
plot_single_run_outputs(epi_model, targets)

In [ ]:
example_matrix_fig = plot_example_model_matrices(epi_model, parameters)

In [ ]:
add_image_to_doc(example_matrix_fig, 'example_matrices', '', app_doc, 'Dynamic mixing matrices')